<a href="https://colab.research.google.com/github/belouspavel/yolov8/blob/main/YOLOv8_ipynb_%D1%81%D0%B2%D0%BE%D0%B4_10_0%2B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основная часть

Реализовано раздельное детектирование и треккинг объектов по уже готовым массивам из детекции, что позволяет подбирать архитектуру трекера и постобработки изолировано, без необходимости запуска предикта YOLO (ускорение подбора треккера и постобработки примерно в 100 раз и без траты лимита GPU в колаб) - также потенциально возможна генетика архитектуры постобработки и треккинга. Кроме того, реализован проброс касок и жилетов мимо трекера, что позволяет лучше трековать людей, а также не приводит к фильтрации касок и жилетов трекером, это позволяет лучше детектировать нарушения, избегать ложных фиксаций нарушений. Также реализовано восстановление в массиве информации нетрекованных боксов людей (которые трекер отфильтровал, отбросил).
В постобработке реализовано два подхода по подсчету людей а также вероятностный подход фиксации нарушений. Под вероятностным понимается отношение количества кадров в которых конкретный айди находится в каске и жилете к общему количеству кадров с этим айди (отношение больше минимального порога - нарушения нет).

У нас рабочий ноутбук, есть закомментированные строки кода. Закомментированные строки кода можно удалить (это вариации кода которые отключены - без них код работает), за исключением сохранения детекции по модели - эта часть кода закомментирована потому что понадобится один, первый раз для сохранения детекций сделанной рассматриваемой вариацией модели.

## Загрузка датасэта

In [ ]:
from google.colab import drive           # у кого автоматически не подключается гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
!pip install filterpy
!pip install imageio-ffmpeg
!git clone https://github.com/noahcao/OC_SORT.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 24.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=0be9b713a7464fee7d8b814515bf97c18c2d243eb03b12fc5cbd9a308788d137
  Stored in directory: /root/.cache/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'OC_SORT'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressi

In [ ]:
from ultralytics import YOLO
%cd OC_SORT
from trackers.ocsort_tracker.ocsort import OCSort
import numpy as np
import pandas as pd
from IPython.display import clear_output
from IPython.display import Image, clear_output  # to display images
from ultralytics.yolo.utils.checks import check_requirements
import matplotlib.pyplot as plt
from moviepy.video.io.VideoFileClip import VideoFileClip
from PIL import Image, ImageDraw, ImageFont
import cv2 
import os
from google.colab import files
from sklearn.metrics import precision_recall_fscore_support
%matplotlib inline

clear_output()

In [ ]:
def get_boxes(result):
  orig_shp = result[0].orig_shape
  all_boxes = np.empty((0, 7))
  for i in range(len(result)):
    bbox = result[i].cpu().boxes.data.numpy()
    bbox = np.hstack((bbox, np.tile(i, (bbox.shape[0], 1))))
    all_boxes = np.vstack((all_boxes, bbox))
  return result, all_boxes, orig_shp

## Обучение и сохранение детекции по набору видео
Эту часть кода можно (нужно) выполнить только один раз для одной модели. Потом можно закоментить, мы будем использовать готовые файлы

In [ ]:
## дообучение 
# model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:01<00:00, 21.7MB/s]


In [ ]:
# model.train(data='/content/drive/MyDrive/dataset-v1.2/data_custom.yaml', epochs=20)  # Обучаем модель

In [ ]:
# from google.colab import files
# files.download('/content/runs/detect/train/weights/best.pt') 

In [ ]:
# path = '/content/drive/MyDrive/best.pt/45/'   ## Модель    
# _best_pt = 'best45.pt'
# for N in range(1,44): # устанавливаем какое видео смотрим
#     try:
#         with open(f'/content/drive/MyDrive/dataset-v1.1/test/{N}.mp4', 'r') as f:
#           model = YOLO(path+_best_pt)  ## каждый раз инициализируем модель в колабе иначе выдает ошибочный результат
#           results, all_boxes, orig_shape = get_boxes(model.predict(f'/content/drive/MyDrive/dataset-v1.1/test/{N}.mp4',
#                                                                 line_thickness = 2,vid_stride = 1, save = True))
#           # np.save(path + f"{N}.npy", np.array((orig_shape, all_boxes)))
#           # df_all_boxes = pd.DataFrame(all_boxes)
#           # df_all_boxes.to_excel(path + f"df_{N}_{orig_shape[1]}_.xlsx")

#     except:    
#         print(f'Видео {N}: отсутствует')

## Трекккинг и постобработка

### Функции для подготовки фреймов данных

In [ ]:
def change_bbox(bbox, tail):
    y2 = bbox[:, [1]] + tail

    bbox[:, [3]] = y2

    return bbox

In [ ]:
def forward(bbox,tracks): # эта функция позволяет сохранить лист детекций в который внесены айди от трека сохраняя нетрекованные боксы на случай последующей перетрековки
  person = np.empty((0, 8))           # Создадим пустой массив для каждого кадра который будем наполнять
  for i, bb in enumerate(bbox):  #Сравним каждый первичный не треккованный бокс
    for k, t in enumerate(tracks):  # С каждым треккованым
      if round(t[0]) == round(bb[0]) and round(t[1]) == round(bb[1]) and round(t[2]) == round(bb[2]) and round(t[3]) == round(bb[3]):   # Если у них совпадают координаты
        bb_tr = np.copy(bb)
        bb_tr = np.insert(bb_tr, 6, t[4])     # Добавляем к нетрекованному боксу трек определенный треккером (таким образом сохраняя конфиденс и класс)
        person = np.vstack((person,bb_tr))        # Складываем массив. На этом этапе остались в стеке только трекованные боксы. Но нам хотелось бы сохранить их все для фиксации нарушений или последующей перетрековки
      else:
        pass
    # if sum(np.in1d(bb[:4],tracks[:,:4])) < 4: # добавим в оттрекованный массив то что треккер отсеял (на случай перетрековки)
    #   person = np.vstack((person,np.insert(bb, 6, -1)))
  
  return person

In [ ]:
# def tracking_on_detect(all_boxes,tracker,orig_shp, tail):
def tracking_on_detect(all_boxes,tracker,orig_shp):
  all_boxes_tr = np.empty((0, 8))
  for i in range(int(max(all_boxes[:, -1]))):
    bbox = all_boxes[all_boxes[:, -1] == i]
    bbox_unif = bbox[np.where(bbox[:,5] != 0)][:,:6]  # отбираем форму и каски в отдельный массив который прокинем мимо трека
    bbox_unif = np.hstack((bbox_unif, np.tile(np.nan, (bbox_unif.shape[0], 1)))) # добавляем столбец с айди нан для касок и жилетов
    bbox_unif = np.hstack((bbox_unif, np.tile(i, (bbox_unif.shape[0], 1)))) # сохраняем номер кадра
    bbox = bbox[np.where(bbox[:,5] == 0)] #в трек идут только люди
    # bbox = change_bbox(bbox, tail)
    tracks = tracker.update(bbox[:, :-2], img_size = orig_shp, img_info = orig_shp) # трекуем людей
    person = forward(bbox,tracks) # эта функция позволяет использовать далее лист детекций в который внесены айди от трека (трек фильтрует и удаляет боксы) 
    all_boxes_tr = np.vstack((all_boxes_tr, person)) # складываем людей в массив
    all_boxes_tr = np.vstack((all_boxes_tr, bbox_unif)) # складываем каски и жилеты в массив
  return all_boxes_tr

In [ ]:
def create_video_with_bbox(bboxes, video_source, video_out): # функция отрисовки боксов на соответсвующем видео
  '''Функция записывает видео с рамками объектов, которые передаются в:
  bboxes - ndarray(x1, y1, x2, y2, conf, class, id, frame),
  если последовательность нарушена надо менять внутри функции.
  Другие обязательные аргументы функции:
  video_source - полный путь до исходного видео, на которое нужно наложить рамки;
  video_out - полный путь вновь создаваемого видео. Путь должен быть,
  а файла - не должно быть'''
  vid_src = cv2.VideoCapture(video_source)
  if vid_src.isOpened():
    # Разрешение кадра
    frame_size = (int(vid_src.get(3)), int(vid_src.get(4)))
    # Количество кадров в секунду
    fps = int(vid_src.get(5))
    # Количество кадров в файле
    len_frm = int(vid_src.get(7))
    # Выходное изображение записываем
    vid_out = cv2.VideoWriter(video_out, cv2.VideoWriter_fourcc(* 'XVID'),
                              fps, frame_size)
    # Пройдемся по всем кадрам
    for i in range(len_frm):
      ret, frame = vid_src.read()
      # На всякий пожарный случай выход
      if not ret: break
      # Отбираем рамки для кадра
      bbox = bboxes[bboxes[:, -1] == i, :-1]
      if len(bbox) > 0:
        # Только люди
        pbox = bbox[bbox[:, 5] == 0]
        for p in pbox:
          # Добавим рамки
          x1, y1 = int(p[0]), int(p[1])
          x2, y2 = int(p[2]), int(p[3])
          cv2.rectangle(frame, (x1, y1), (x2, y2), (153,153,153), 2)
          # Добавим надпись в виде идентификатора объекта  и conf
          msg = 'id' + str(int(p[6])) + ' ' + str(round(p[4], 2))
          (w, h), _ = cv2.getTextSize(msg, cv2.FONT_HERSHEY_DUPLEX, 0.45, 1)
          cv2.rectangle(frame, (x1, y1 - 20), (x1 + w, y1), (153,153,153), -1)
          cv2.putText(frame, msg, (x1, y1 - 5), cv2.FONT_HERSHEY_DUPLEX, 0.45,
                      (255,255,255), 1)
        # Отрисовываем рамки касок
        helmets = bbox[bbox[:, 5] == 1]
        for helmet in helmets:
          x1, y1 = int(helmet[0]), int(helmet[1])
          x2, y2 = int(helmet[2]), int(helmet[3])
          cv2.rectangle(frame, (x1, y1), (x2, y2), (51,255,204), 2)
        # Отрисовываем рамки жилетов
        vests = bbox[bbox[:, 5] == 2]
        for vest in vests:
          x1, y1 = int(vest[0]), int(vest[1])
          x2, y2 = int(vest[2]), int(vest[3])
          cv2.rectangle(frame, (x1, y1), (x2, y2), (51,204,255), 2)
      # Записываем кадр
      vid_out.write(frame)
    # Освобождаем видео
    vid_out.release()
    vid_src.release()
  else: print('Видеофайл недоступен')
  # Заклинание cv2
  cv2.destroyAllWindows()

In [ ]:
def get_men(out_boxes):
  men = np.empty((0, 9))
  inter_helm = 60 # поле вокруг бб человека для детекции касок 
  inter_unif = 15 # поле вокруг бб человека для детекции жилетов
  for i in range(int(max(out_boxes[:, -1]))):
   
    # Только люди
    humans = out_boxes[(out_boxes[:, -3] == 0) & (out_boxes[:, -2] != -1) & (out_boxes[:, -1] == i)]
    # Только каски
    helmets = out_boxes[(out_boxes[:, -3] == 1) & (out_boxes[:, -1] == i)]
    # Только жилетки
    vests = out_boxes[(out_boxes[:, -3] == 2) & (out_boxes[:, -1] == i)]

    # Персональный подход к каждому человеку
    for man in humans:
      # Сколько касок в пределах рамок человека (либо 1, либо 0)
      helmet = 1 if len(helmets[(helmets[:, 0] >= man[0] - inter_helm) & (helmets[:, 1] >= man[1] - inter_helm) & 
                                (helmets[:, 2] <= man[2] + inter_helm) & (helmets[:, 3] <= man[3] + inter_helm)]) >= 1 else 0
      # Сколько жилеток в пределах рамок человека (либо 1, либо 0)
      vest = 1 if len(vests[(vests[:, 0] >= man[0] - inter_unif) & (vests[:, 1] >= man[1] - inter_unif) & 
                            (vests[:, 2] <= man[2] + inter_unif) & (vests[:, 3] <= man[3] + inter_unif)]) >= 1 else 0
      #Это условие нужно для начала работы, потому что несколько первых кадров 
      #не имеют идентификатора. Можно отбросить первые кадры, здесь так и делается
        # Это просто добавление в массив men. Часть параметров нужны нам дважды для выявления макс и мин.
        # Поэтому дважды повторяются ордината низа и номер кадра
      men = np.vstack((men, np.array([man[-2], 
                                        man[1], man[1],
                                        man[3], man[3], 
                                        i, i,
                                        helmet, vest])))
      # Формируем датафрейм. Будем считать низ вначале и в конце, также первый кадр
      # и последний кадр, где был этот id (пока это одно и тоже значение)
  return men

In [ ]:
# def get_count_men(n_count, men, orig_shape):# принимаем каличество кадров без детекции для переназначения id из Optuna
# def get_count_men(men, orig_shape, barier, buff, tail):# принимаем барьериз Optuna
# def get_count_men(tail, men, orig_shape):# определяем все сами
# def get_count_men(men, orig_shape, buff):
def get_count_men(men, orig_shape):# определяем все сами
  n_ = int(max(men[:,0]))
  orig_shape = int(orig_shape)
  incoming = 0                                         # количество вошедших
  exiting = 0                                        # количество вышедших
  barier = 337
  
  gate_y = barier * orig_shape / 640 # определяем барьер сами


  box_y_top     = [None] + [list() for _ in range(int(n_))]               
  box_y_bottom  = [None] + [list() for _ in range(int(n_))]               
  box_frame     = [None] + [list() for _ in range(int(n_))]
               
  


  for i, m in enumerate(men):
    id = int(m[0])
    frame_n = int(m[-4])
    y_top = float(m[2])
    y_bottom = float(m[4])
    box_y_top[id].append(y_top)
    box_y_bottom[id].append(y_bottom)
    box_frame[id].append(frame_n)

    if len(box_frame[id]) > 11: # принудительно переназначаем айди на входе и выходе если треккер не переназначил
      condition = box_frame[id][-1] - box_frame[id][-2] > 10 and ((box_y_top[id][-1] / orig_shape < 0.2) or (box_y_bottom[id][-1] / orig_shape > 0.8))
                # условие смены id: 
                # верхняя граница бб в верхней части кадра  или нижняя граница бб в нижней части кадра
                # бб не детектировался в течение 20 предыдущих кадров                             
      if condition: 
        n_ += 1
        box_y_top.append([])
        box_y_bottom.append([])
        box_frame.append([])
        for j in range(i, len(men)):       
          if men[j][0] == id:
            men[j][0] = n_

  if len(box_frame[id]) < 21: # удаляем айди чей трек короче n кадров
    men = men[~np.isin(men[:,0], id)]

  
  n = int(max(men[:,0]))

  human_c = [None] + [list() for _ in range(int(n))] 
  for m in men:
    num = int(m[0])
    box_center = (float(m[4]) - float(m[2]))/2+float(m[2])

    # if float(m[1]) + tail* orig_shape / 640 < 640 * orig_shape / 640:
    #   box_center = float(m[1]) +tail * orig_shape / 640
    # else:
    #   box_center = 640 * orig_shape / 640   

    # box_center = float(m[1]) #+ 250.0 # берем не истинный центр бб а относительно головы потому что такой центр более устойчив и не прыгает
    human_c[num].append(box_center)

  ind = []
  for i, h in enumerate(human_c):   

    if h and h[0] < gate_y  < h[-1]:
      incoming += 1
      print(i, "Вошел") #отладка
      ind.append(i)
    elif h and h[0] > gate_y > h[-1]:
      exiting += 1
      print(i, "Вышел") #отладка
      ind.append(i)

    # if h and h[0] < gate_y  < h[-1] or h and h[0] < (gate_y-buff)  < h[-1]:
    #   incoming += 1
    # elif h and h[0] > gate_y > h[-1] or h and h[0] > (gate_y-buff) > h[-1]:
    #   exiting += 1
  men = men[np.isin(men[:,0], ind)]

  
  return men, incoming, exiting

In [ ]:
def get_count_vialotion(men, orig_shape): # step height определяем сами
# def get_count_vialotion(men, orig_shape,step,height): # step height из optuna
  orig_shape = int(orig_shape)
  # format = orig_shape/640
  # step = 46
  # height = 200
  df = pd.DataFrame(men, columns=['id', 
                                  'first_top_y', 'last_top_y', 
                                  'first_bottom_y','last_bottom_y', 
                                  'first_frame', 'last_frame', 
                                  'helmet', 'uniform'])
  # Зададим правила агрегирующих функций
  agg_func = {'first_top_y': 'first', 'last_top_y': 'last',
              'first_bottom_y': 'first','last_bottom_y': 'last', 
              'first_frame': 'first', 'last_frame': 'last',
              'helmet':'mean', 'uniform': 'mean' 
              }
  # Группируем по id
  df1 = df.groupby('id').agg(agg_func)
  # Чтобы не выводилось предупреждение
  pd.options.mode.chained_assignment = None
  # Определяем пройденное расстояние
  # df1.loc[:, 'last_cent_y_stand'] = df1.last_top_y+height*format
  # df1.loc[df1['last_cent_y_stand'] > orig_shape, "last_cent_y_stand"] = orig_shape # здесь считаем низ от макушки а не по низу бокса, фиксируем, но если он ниже размера картинки то лимитируем по низу картинки
  # df1.loc[:, 'first_cent_y_stand'] = df1.first_top_y+height*format
  # df1.loc[df1['first_cent_y_stand'] > orig_shape, "first_cent_y_stand"] = orig_shape # здесь считаем низ от макушки а не по низу бокса, фиксируем, но если он ниже размера картинки то лимитируем по низу картинки
  df1.loc[:, 'distance'] = df1.last_bottom_y - df1.first_bottom_y
  # df1.loc[:, 'distance'] = df1.last_bottom_y - df1.first_bottom_y
  # df2 = df1.loc[(np.abs(df1.distance) > (step*format))] #здесь если человек вошел к турникету и вернулся ко входу то его дистанция будет около нуля и он отсеется, также как и охранник в обратном случае
  df2 = df1.copy()
  # Считаем входящих (сверху вниз)
  incoming = df2.loc[df2.distance > 0].shape[0]
  # Считаем выходящих
  exiting = df2.loc[df2.distance < 0].shape[0]

  V_helm = 0.15
  V_unif = 0.5
  dictinex = {'incoming':incoming , 'exiting': exiting}
  df2.loc[df2.helmet < V_helm, 'helmet'] = 0
  df2.loc[df2.helmet >= V_helm, 'helmet'] = 1
  df2.loc[df2.uniform < V_unif, 'uniform'] = 0
  df2.loc[df2.uniform >= V_unif, 'uniform'] = 1
  
  clothing_helmet =[] # соберем данные по одежде в отдельный массив для последующей проверки на этапе проверки P и R
  clothing_unif =[]
  for i, ds in enumerate(df2.values):
    clothing_helmet.append(int(ds[6]))
    clothing_unif.append(int(ds[7]))
  violations = df2.loc[((df2.helmet == 0) | (df2.uniform == 0)), 
                     ['helmet', 'uniform', 'first_frame', 'last_frame']]   # а это сами нарушения с номерами кадров
  # Можно отдельно отобрать три группы нарушителей, но здесь все вместе
  # Пока только вывод на экран первых, можно сохранить в csv
  # return men
  return violations, incoming, exiting, df2, clothing_helmet, clothing_unif

In [ ]:
# Отладка
path = '/content/drive/MyDrive/best.pt/45/'   ## Модель
tracker_path = 'ocsort00/'
if not os.path.isdir(path+tracker_path):
  os.makedirs(path+tracker_path)
video_source ='/content/drive/MyDrive/dataset-v1.1/test/'

tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, # # 45 модель 2 ошибки
                  min_hits = 7, iou_threshold = 0.6247364818234254, 
                  delta_t = 5, asso_func = 'iou', 
                  inertia = 0.6758806605183052, use_byte = True)

for N in range(6,7): # устанавливаем какое видео смотрим
  with open(path +f'{N}.npy', 'rb') as files:  #Загружаем объект содержащий формат исходного изображения и детекции
    all_boxes_and_shp = np.load(files, allow_pickle=True)
    orig_shp = all_boxes_and_shp[0] # Здесь формат
    all_boxes = all_boxes_and_shp[1]  # Здесь боксы
    out_boxes = tracking_on_detect(all_boxes,tracker, orig_shp)   # Отправляем боксы в трекинг + пробрасываем мимо трекинга каски и нетрекованные боксы людей
    create_video_with_bbox(out_boxes, video_source + f'{N}.mp4', path + tracker_path + f'{N}_track.mp4') # функция отрисовки боксов на соответсвующем видео
    # out_boxes_pd = pd.DataFrame(out_boxes)
    men = get_men(out_boxes)   # Смотрим у какого айди есть каски и жилеты (по порогу от доли кадров где был зафиксирован айди человека + каска и жилет в его бб и без них)
    men_clean, incoming1, exiting1 = get_count_men(men, orig_shp[1]) 
    # violation, incoming2, exiting2,df, clothing_helmet, clothing_unif = get_count_vialotion(men_clean, orig_shp[1]) # Здесь принимаем переназначенные айди смотрим нарушения а также повторно считаем входящих по дистанции, проверяем


3 Вышел
6 Вышел
7 Вышел
10 Вышел
14 Вошел


### Подсчет количества людей

In [ ]:
#Истинные значения вошедших и вышедших на видео
dict_in_true =  {'1':3,'2':3,'3':3,'4':4,'5':0,'6':1,'7':0,'8':0,'9':1,'10':0,'11':0,'12':0,'13':2,'14':0,'15':2,'16':0,'17':0,'18':4,'19':1,'20':0,'21':1,'22':0,'23':0,'24':1,'25':2,'26':0,'27':1,'28':1,'29':1,'30':1,'31':0,'32':0,'33':0,'34':0,'35':0,'36':0,'37':0,'38':8,'39':0,'40':0,'41':0,'42':2,'43':0}  # 20 видео битое с невошедшим призраком (человек раздвоился)
dict_out_true = {'1':0,'2':0,'3':0,'4':0,'5':0,'6':4,'7':4,'8':5,'9':1,'10':2,'11':3,'12':4,'13':4,'14':3,'15':0,'16':4,'17':6,'18':0,'19':0,'20':0,'21':1,'22':3,'23':1,'24':0,'25':0,'26':1,'27':1,'28':1,'29':1,'30':3,'31':2,'32':0,'33':0,'34':0,'35':0,'36':0,'37':0,'38':7,'39':1,'40':1,'41':0,'42':0,'43':0}

In [ ]:
def compare_dict(d1,d2): #функция сравнения словарей входящих/выходящих предсказанных с эталонным словарем
  dict_all ={}
  d_1 = set(d1.keys())
  d_2 = set(d2.keys())
  common_keys = d_1.intersection(d_2)
  
  if common_keys:
    dict_all = {k: d1[k] - d2[k] for k in common_keys}
  return dict_all

In [ ]:
path = '/content/drive/MyDrive/best.pt/45/'   ## Модель
tracker_path = 'ocsort00/'
if not os.path.isdir(path+tracker_path):
  os.makedirs(path+tracker_path)
video_source ='/content/drive/MyDrive/dataset-v1.1/test/' # папка с исходными видео для отрисовки готовых боксов

tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, # # 45 модель 2 ошибки
                  min_hits = 7, iou_threshold = 0.6247364818234254, 
                  delta_t = 5, asso_func = 'iou', 
                  inertia = 0.6758806605183052, use_byte = True)


# # # создаем пустые словари которые будем наполнять предсказаниями вошедших /вышедших первым  и вторым алгоритмом
d_in1 = {str(n): 0 for n in list(range(1,44))} 
d_out1 = {str(n): 0 for n in list(range(1,44))}
d_in2 = {str(n): 0 for n in list(range(1,44))} 
d_out2 = {str(n): 0 for n in list(range(1,44))}

# # создаем пустые словари которые будем наполнять предсказаниями нарушениями по каске (используются в следующем разделе)
# # создаем пустые словари которые будем наполнять предсказаниями нарушениями по униформе (используются в следующем разделе)
d_helmet = {str(n): [] for n in list(range(1,44))} 
d_unif = {str(n): [] for n in list(range(1,44))} 

for N in range(1,44): # устанавливаем какое видео смотрим
    try:
        with open(path +f'{N}.npy', 'rb') as files:  #Загружаем объект содержащий формат исходного изображения и детекции
          all_boxes_and_shp = np.load(files, allow_pickle=True)
          orig_shp = all_boxes_and_shp[0] # Здесь формат
          all_boxes = all_boxes_and_shp[1]  # Здесь боксы
          out_boxes = tracking_on_detect(all_boxes,tracker, orig_shp)   # Отправляем боксы в трекинг + пробрасываем мимо трекинга каски и нетрекованные боксы людей
          create_video_with_bbox(out_boxes, video_source + f'{N}.mp4', path + tracker_path + f'{N}_track.mp4') # функция отрисовки боксов на соответсвующем видео
          # out_boxes_pd = pd.DataFrame(out_boxes)
          # out_boxes_pd.to_excel(path + tracker_path + f"df_{N}_{round(orig_shp[1])}_.xlsx") # сохраняем что бы было)
          men = get_men(out_boxes)   # Смотрим у какого айди есть каски и жилеты (по порогу от доли кадров где был зафиксирован айди человека + каска и жилет в его бб и без них)
          men_clean, incoming1, exiting1 = get_count_men(men, orig_shp[1]) # здесь переназначаем айди входящий/выходящий (временное решение для MVP, надо думать над продом)
          violation, incoming2, exiting2,df, clothing_helmet, clothing_unif = get_count_vialotion(men_clean, orig_shp[1]) # Здесь принимаем переназначенные айди смотрим нарушения а также повторно считаем входящих по дистанции, проверяем

          d_in1[f'{N}'] = incoming1
          d_out1[f'{N}'] = exiting1
          d_in2[f'{N}'] = incoming2
          d_out2[f'{N}'] = exiting2
          d_helmet[f'{N}'] = clothing_helmet         
          d_unif[f'{N}'] = clothing_unif       

    except:    
        print(f'данные по видео {N}: отсутствуют')

# # Далее модуль распечатки данных
d_in_comp = compare_dict(d_in1, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in1[key]}')
print(f'суммарное количество ошибок первого алгоритма на входящих  {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out1, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out1[key]}')
print(f'суммарное количество ошибок первого алгоритма на выходящих {sum(abs(ele) for ele in d_out_comp.values())}')

d_in_comp = compare_dict(d_in2, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in2[key]}')
print(f'суммарное количество ошибок второго алгоритма на входящих  {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out2, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out2[key]}')
print(f'суммарное количество ошибок второго алгоритма на выходящих {sum(abs(ele) for ele in d_out_comp.values())}')

d_in_comp = compare_dict(d_in2, dict_in_true) #суммарное количество ошибок первого алгоритма на входящих  
d_out_comp = compare_dict(d_out2, dict_out_true) # суммарное количество ошибок первого алгоритма на выходящих 
d_comp = sum(abs(ele) for ele in d_in_comp.values()) + sum(abs(ele) for ele in d_out_comp.values())

данные по видео 32: отсутствуют
данные по видео 33: отсутствуют
данные по видео 34: отсутствуют
данные по видео 35: отсутствуют
данные по видео 37: отсутствуют
данные по видео 41: отсутствуют
данные по видео 43: отсутствуют
суммарное количество ошибок первого алгоритма на входящих  0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок первого алгоритма на выходящих 2
суммарное количество ошибок второго алгоритма на входящих  0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок второго алгоритма на выходящих 2


### Оптимизация  кол-ва людей Optuna

In [ ]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import optuna

optuna.__version__

'3.1.1'

In [ ]:
# yaml2 ={  asso_func: giou
#   conf_thres: 0.5122620708221085
#   delta_t: 1
#   det_thresh: 0
#   inertia: 0.3941737016672115
#   iou_thresh: 0.22136877277096445
#   max_age: 50
#   min_hits: 1
#   use_byte: false} # микель 1

In [ ]:
# # 45 модель 2 алгоритм 6 ошибок
  # tracker = OCSort(det_thresh = 0.5972122815852265, max_age = 7,
  #                 min_hits = 5, iou_threshold = 0.1330893701047955, 
  #                 delta_t = 2, asso_func = 'iou', 
  #                 inertia = 0.2407496403510045, use_byte = False)

In [ ]:
# # 45 модель 1 алгоритм барьер 337 2 ошибки
  # tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, # # 45 модель 2 ошибки
  #                 min_hits = 7, iou_threshold = 0.6247364818234254, 
  #                 delta_t = 5, asso_func = 'iou', 
  #                 inertia = 0.6758806605183052, use_byte = True)
# {'det_thresh': 0.4862688342603671, 'max_age': 7.456359330880515, 'min_hits': 6.084068672838238, 'iou_threshold': 0.6268883790023444, 'delta_t': 5.984993700881928, 'asso': 'iou', 'inertia': 0.6557787153239634, 'use_byte': True, 'barier': 336}. 
# {'det_thresh': 0.46648242342039237, 'max_age': 7.203196769474859, 'min_hits': 6.433879332154186, 'iou_threshold': 0.6305934741741774, 'delta_t': 5.381967701269865, 'asso': 'iou', 'inertia': 0.6369425863377747, 'use_byte': True, 'barier': 337}. 
# {'det_thresh': 0.49428431641933235, 'max_age': 6.910533298012, 'min_hits': 6.535011489592976, 'iou_threshold': 0.6247364818234254, 'delta_t': 5.248070195914983, 'asso': 'iou', 'inertia': 0.6758806605183052, 'use_byte': True, 'barier': 340}. 
# {'det_thresh': 0.4800010924947306, 'max_age': 7.616682342988325, 'min_hits': 6.235897942703035, 'iou_threshold': 0.6203530097118262, 'delta_t': 5.516049204278155, 'asso': 'iou', 'inertia': 0.6463059396477179, 'use_byte': True, 'barier': 339}. 
# {'det_thresh': 0.4581106410893448, 'max_age': 7.0674184728269385, 'min_hits': 6.234815471224487, 'iou_threshold': 0.6290289995639651, 'delta_t': 5.034286694675483, 'asso': 'iou', 'inertia': 0.6385260183285119, 'use_byte': True, 'barier': 339}. 
# {'det_thresh': 0.46225643518026666, 'max_age': 7.036458426921717, 'min_hits': 6.3261509842693116, 'iou_threshold': 0.6262538211501386, 'delta_t': 5.003513218268599, 'asso': 'iou', 'inertia': 0.6287646232751898, 'use_byte': True, 'barier': 338}. 
# # 45 модель 4 ошибки
# {'det_thresh': 0.5413427615421595, 'max_age': 7.379847519608593, 'min_hits': 6.804953426768252, 'iou_threshold': 0.6406466166134259, 'delta_t': 6.955562247157338, 'asso': 'iou', 'inertia': 0.7246055201833036, 'use_byte': True, 'barier': 335}
# {'det_thresh': 0.3776929765510213, 'max_age': 7.7378061293440785, 'min_hits': 6.40569787806849, 'iou_threshold': 0.6468340994720151, 'delta_t': 5.398816334493228, 'asso': 'iou', 'inertia': 0.6616686833612563, 'use_byte': True, 'barier': 334}

In [ ]:
def objective(trial):
  path = '/content/drive/MyDrive/best.pt/45/'   ## Модель   
  path = path
  tracker_path = '/ocsort00/' # default
  seed = 123
  # det_thresh = trial.suggest_float('det_thresh', 0.05, 0.95, log=True) # ШИРОКИЙ ДИАПАЗОН
  # max_age= int(trial.suggest_float('max_age', 5, 70, log=True)) # 30
  # min_hits= int(trial.suggest_float('min_hits', 1, 10, log=True)) # 1
  # iou_threshold = trial.suggest_float('iou_threshold', 0.05, 0.95, log=True) 
  # delta_t= int(trial.suggest_float('delta_t', 1, 10, log=True)) # 1  
  # asso_func= trial.suggest_categorical('asso_func', ["iou","giou","ciou","diou","ct_dist"]) #
  # inertia=trial.suggest_float('inertia', 0.05, 0.95, log=True) # 
  # use_byte=trial.suggest_categorical('use_byte', [False,True]) #
  # step = trial.suggest_int('step', 20, 150, log=True)
  # height = trial.suggest_int('height', 100, 350, log=True)


  
  # det_thresh = trial.suggest_float('det_thresh', 0.36, 0.56, log=True) # узкий диапазон на 45 модель 
  # max_age= int(trial.suggest_float('max_age', 6, 8, log=True)) # 
  # min_hits= int(trial.suggest_float('min_hits', 6, 8, log=True)) # 
  # iou_threshold = trial.suggest_float('iou_threshold', 0.62, 0.66, log=True) 
  # delta_t= int(trial.suggest_float('delta_t', 5, 8, log=True)) #   
  # asso_func= trial.suggest_categorical('asso_func', ["iou"]) 
  # inertia=trial.suggest_float('inertia', 0.6, 0.75, log=True) 
  # use_byte=trial.suggest_categorical('use_byte', [True]) 
  # barier = trial.suggest_int('barier', 330, 340, log=True)


  step = trial.suggest_int('step', 15, 60, log=True)
  height = trial.suggest_int('height', 200, 450, log=True)  
  # barier = trial.suggest_int('barier', 300, 400, log=True)
  # buff = trial.suggest_int('buff', 5, 70, log=True)
  # tail = trial.suggest_int('tail', 150, 250, log=True)

  
  tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, # # 45 модель 2 ошибки
                  min_hits = 7, iou_threshold = 0.6247364818234254, 
                  delta_t = 5, asso_func = 'iou', 
                  inertia = 0.6758806605183052, use_byte = True)

  # tracker = OCSort(det_thresh = det_thresh,max_age = max_age, min_hits = min_hits,
  #                  iou_threshold = iou_threshold, delta_t = delta_t, asso_func= asso_func,
  #                  inertia = inertia, use_byte = use_byte)



  # создаем пустые словари которые будем наполнять предсказаниями вошедших /вышедших первым  и вторым алгоритмом
  d_in1 = {str(n): 0 for n in list(range(1,44))} 
  d_out1 = {str(n): 0 for n in list(range(1,44))}
  d_in2 = {str(n): 0 for n in list(range(1,44))} 
  d_out2 = {str(n): 0 for n in list(range(1,44))}

  # создаем пустые словари которые будем наполнять предсказаниями нарушениями по каске (используются в следующем разделе)
  # создаем пустые словари которые будем наполнять предсказаниями нарушениями по униформе (используются в следующем разделе)
  d_helmet = {str(n): [] for n in list(range(1,44))} 
  d_unif = {str(n): [] for n in list(range(1,44))} 

  for N in range(1,44): # устанавливаем какое видео смотрим
      try:
          with open(path+f'{N}.npy', 'rb') as files:  #Загружаем объект содержащий формат исходного изображения и детекции
            all_boxes_and_shp = np.load(files, allow_pickle=True)
            orig_shp = all_boxes_and_shp[0] # Здесь формат
            all_boxes = all_boxes_and_shp[1]  # Здесь боксы
            out_boxes, bbox = tracking_on_detect(all_boxes,tracker, orig_shp)   # Отправляем боксы в трекинг + пробрасываем мимо трекинга каски и нетрекованные боксы людей
            out_boxes_pd = pd.DataFrame(out_boxes)
            # out_boxes_pd.to_excel(path + tracker_path + f"df_{N}_{round(orig_shp[1])}_.xlsx") # сохраняем что бы было)
            men = get_men(out_boxes)   # Смотрим у какого айди есть каски и жилеты (по порогу от доли кадров где был зафиксирован айди человека + каска и жилет в его бб и без них)
            men_clean, incoming1, exiting1 = get_count_men(men, orig_shp[1]) # здесь переназначаем айди входящий/выходящий (временное решение для MVP, надо думать над продом)
            violation, incoming2, exiting2,df, clothing_helmet, clothing_unif = get_count_vialotion(men_clean, orig_shp[1],step,height) # Здесь принимаем переназначенные айди смотрим нарушения а также повторно считаем входящих по дистанции, проверяем

            d_in1[f'{N}'] = incoming1
            d_out1[f'{N}'] = exiting1
            d_in2[f'{N}'] = incoming2
            d_out2[f'{N}'] = exiting2
            d_helmet[f'{N}'] = clothing_helmet         
            d_unif[f'{N}'] = clothing_unif       

      except:    
          pass

    # Далее модуль распечатки данных
  d_in_comp = compare_dict(d_in2, dict_in_true) #суммарное количество ошибок первого алгоритма на входящих  
  d_out_comp = compare_dict(d_out2, dict_out_true) # суммарное количество ошибок первого алгоритма на выходящих 
  d_comp = sum(abs(ele) for ele in d_in_comp.values()) + sum(abs(ele) for ele in d_out_comp.values())
  # print(f'суммарное количество ошибок первого алгоритма  {d_comp}')
  return d_comp

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

trial = study.best_trial

print('Количество нарушений минимальное: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-04-11 12:17:27,427] A new study created in memory with name: no-name-33677a6a-f390-4979-9ce2-99dbc1b8ea71
[I 2023-04-11 12:18:13,538] Trial 0 finished with value: 11.0 and parameters: {'step': 15, 'height': 415}. Best is trial 0 with value: 11.0.


### Plotting the study

Plotting the optimization history of the study.

In [ ]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [ ]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [ ]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])

### Нарушения на видео

In [ ]:
#Истинные значения нарушений на видео
d_true_helmet = {'1': [1, 1, 1], '2': [0, 1, 1], '3': [1, 0, 1], '4': [1, 0, 1, 1], '5': [], '6': [0, 0, 0, 0, 1], '7': [0, 0, 0, 0], '8': [0, 0, 0, 0, 0], '9': [0, 1], '10': [0, 0], '11': [0, 0, 0], '12': [0, 0, 0, 1], '13': [1, 0, 0, 0, 0, 0], '14': [1, 1, 1], '15': [0, 0], '16': [0, 0, 1, 0], '17': [0, 1, 0, 0, 0, 0], '18': [1, 1, 1, 1], '19': [0], '20': [], '21': [0,0], '22': [0, 1, 0], '23': [0], '24': [1], '25': [0, 0], '26': [0], '27': [0, 1], '28': [0, 1], '29': [0, 0], '30': [0, 0, 0, 0], '31': [1, 1], '32': [], '33': [], '34': [], '35': [], '36': [], '37': [], '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1], '39': [1], '40': [0], '41': [], '42': [0, 0], '43': []} # 20 видео битое с невошедшим призраком (человек раздвоился)
d_true_unif = {'1': [1, 1, 1], '2': [1, 1, 1], '3': [1, 0, 1], '4': [1, 0, 1, 1], '5': [], '6': [0, 0, 0, 0, 1], '7': [1, 0, 1, 0], '8': [0, 0, 0, 0, 0], '9': [1, 1], '10': [1, 1], '11': [0, 0, 1], '12': [0, 0, 0, 1], '13': [1, 1, 0, 0, 1, 1], '14': [1, 1, 1], '15': [0, 0], '16': [1, 1, 1, 0], '17': [0, 1, 0, 0, 1, 1], '18': [1, 1, 0, 0], '19': [1], '20': [], '21': [0,0], '22': [0, 1, 0], '23': [0], '24': [1], '25': [0, 0], '26': [0], '27': [0, 1], '28': [0, 1], '29': [0, 0], '30': [0, 1, 1, 1], '31': [1, 1], '32': [], '33': [], '34': [], '35': [], '36': [], '37': [], '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1], '39': [1], '40': [0], '41': [], '42': [0, 0], '43': []} # 20 видео битое с невошедшим призраком (человек раздвоился)

In [ ]:
""" 
Эта функция выравнивает длину массива в нарушениях по каждому видео так что бы подать в метрику sklearn массивы одной длины
Делает она это следующим образом: 
- Создаются копии массивов с эталоном и предсказанием
- Изменению подвергаются только те массивы (их копии) по видео в которых количество айди предсказанных отличается от эталонного !!!(их всего 1-2)!!!
В других массивах ничего менять не нужно потому что порядок в котором айди попадают в предсказания совпадает с эталоном 
(проверено в ручную на нескольких экспериментах)
- Копии этих массивов сортируются от 1 к 0 и там где не хватает значений добавляются маркеры отсутсвия детекции или доп детекции в случае эталона
( двойки - это третий класс снижающий P R для наших двух нарушения/ненарушения) 
Таким образом мы выравниваем длины массивов


Идея заключается в том что нам нужно при вычислении метрик подать пары значений и главное подать ту пару где у нас происходит отличие предсказания от эталона
Т.е. если предсказание определило 3 человека в касках а реально их было 4, то мы добьем четвертое значение двойкой и не важно какой из них не задетектровался
!!!Ситуация при которой два человека в эталоне по форме определились предиктом как один одетый по форме также будет штрафоваться добавлением двойки к массиву т.к. 
в противном случае это могло быть и два человека одетых по разному (один правильно - а другой нет) и обработка этого могла не заметить!!! 

Все обратные логики также справедливы!

Подход в рамках имеющихся тестовых видео работает (проверялось вручную на нашей пособработке по всем видео)

Возможно для цели подбора треккера с максимальной P метрикой по единицам логику нужно перевернуть и сортировать от меньшего к большему, добивать единицами - !Вернуться!

Ниже после формулы приведена демонстрация
"""

def dict_fill(d_pred_, d_true_):  # функция которая выравнивает содержимое в предсказанном словаре с эталоно заполняя его 0
  d_pred_exp = d_pred_.copy()  # создаем новые словари
  d_true_exp = d_true_.copy()
  if len(d_pred_exp) != len(d_true_exp): # если длина значений словарей не совпадает делаем соритрировку от 1 к 0 и добиваем  значения словаря меньшео размера маркером отсутсвия детекции - 2
    d_pred_exp.sort(reverse = True)
    d_true_exp.sort(reverse = True)
    _list = [d_pred_exp,d_true_exp]
    for a in _list:
      a.extend([2] * (max(map(len, _list)) - len(a)))

  return d_pred_exp, d_true_exp

### Демонстрация!
d_pred = {'1': [1, 0, 0], '2': [1, 0, 0, 0, 1, 0],  '3': [1, 0],        '4': [1, 0, 1]} # 1 - одного одетого задетектил как двоих раздетых 2 - лишние треки в середине видео 3 - пропуск отсутсвие детекции в конце 4 -объеденил в середине
d_true = {'1': [1, 1],    '2': [1, 0, 1, 0],        '3': [1, 0, 0, 1],  '4': [1, 0, 0, 1]}
for i in range(1,5):
  d_pred[f'{i}'], d_true[f'{i}'] = dict_fill(d_pred[f'{i}'], d_true[f'{i}'])
print(d_pred)
print(d_true)

{'1': [1, 0, 0], '2': [1, 1, 0, 0, 0, 0], '3': [1, 0, 2, 2], '4': [1, 1, 0, 2]}
{'1': [1, 1, 2], '2': [1, 1, 0, 0, 2, 2], '3': [1, 1, 0, 0], '4': [1, 1, 0, 0]}


In [ ]:
path = '/content/drive/MyDrive/best.pt/45/'   ## Модель   
path = path
tracker_path = '/ocsort00/' # default

tracker = OCSort(det_thresh = 0.49428431641933235, max_age = 7, # # 45 модель 2 ошибки
                  min_hits = 7, iou_threshold = 0.6247364818234254, 
                  delta_t = 5, asso_func = 'iou', 
                  inertia = 0.6758806605183052, use_byte = True)


# # # создаем пустые словари которые будем наполнять предсказаниями вошедших /вышедших первым  и вторым алгоритмом
d_in1 = {str(n): 0 for n in list(range(1,44))} 
d_out1 = {str(n): 0 for n in list(range(1,44))}
d_in2 = {str(n): 0 for n in list(range(1,44))} 
d_out2 = {str(n): 0 for n in list(range(1,44))}

# # создаем пустые словари которые будем наполнять предсказаниями нарушениями по каске (используются в следующем разделе)
# # создаем пустые словари которые будем наполнять предсказаниями нарушениями по униформе (используются в следующем разделе)
d_helmet = {str(n): [] for n in list(range(1,44))} 
d_unif = {str(n): [] for n in list(range(1,44))} 

for N in range(1,44): # устанавливаем какое видео смотрим
    try:
        with open(path +f'{N}.npy', 'rb') as files:  #Загружаем объект содержащий формат исходного изображения и детекции
          all_boxes_and_shp = np.load(files, allow_pickle=True)
          orig_shp = all_boxes_and_shp[0] # Здесь формат
          all_boxes = all_boxes_and_shp[1]  # Здесь боксы
          out_boxes,bbox = tracking_on_detect(all_boxes,tracker, orig_shp)   # Отправляем боксы в трекинг + пробрасываем мимо трекинга каски и нетрекованные боксы людей
          out_boxes_pd = pd.DataFrame(out_boxes)
          # out_boxes_pd.to_excel(path + tracker_path + f"df_{N}_{round(orig_shp[1])}_.xlsx") # сохраняем что бы было)
          men = get_men(out_boxes)   # Смотрим у какого айди есть каски и жилеты (по порогу от доли кадров где был зафиксирован айди человека + каска и жилет в его бб и без них)
          men_clean, incoming1, exiting1 = get_count_men(men, orig_shp[1]) # здесь переназначаем айди входящий/выходящий (временное решение для MVP, надо думать над продом)
          violation, incoming2, exiting2,df, clothing_helmet, clothing_unif = get_count_vialotion(men_clean, orig_shp[1]) # Здесь принимаем переназначенные айди смотрим нарушения а также повторно считаем входящих по дистанции, проверяем

          d_in1[f'{N}'] = incoming1
          d_out1[f'{N}'] = exiting1
          d_in2[f'{N}'] = incoming2
          d_out2[f'{N}'] = exiting2
          d_helmet[f'{N}'] = clothing_helmet         
          d_unif[f'{N}'] = clothing_unif       

    except:    
        print(f'данные по видео {N}: отсутствуют')

# # Далее модуль распечатки данных
d_in_comp = compare_dict(d_in1, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in1[key]}')
print(f'суммарное количество ошибок первого алгоритма на входящих  {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out1, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out1[key]}')
print(f'суммарное количество ошибок первого алгоритма на выходящих {sum(abs(ele) for ele in d_out_comp.values())}')

d_in_comp = compare_dict(d_in2, dict_in_true)
for key, value in d_in_comp.items(): 
  if value != 0:
    print(f'в видео {key} входящих {dict_in_true[key]} а не {d_in2[key]}')
print(f'суммарное количество ошибок второго алгоритма на входящих  {sum(abs(ele) for ele in d_in_comp.values())}')

d_out_comp = compare_dict(d_out2, dict_out_true)
for key, value in d_out_comp.items(): 
  if value != 0:
    print(f'в видео {key} выходящих {dict_out_true[key]} а не {d_out2[key]}')
print(f'суммарное количество ошибок второго алгоритма на выходящих {sum(abs(ele) for ele in d_out_comp.values())}')



данные по видео 32: отсутствуют
данные по видео 33: отсутствуют
данные по видео 34: отсутствуют
данные по видео 35: отсутствуют
данные по видео 37: отсутствуют
данные по видео 41: отсутствуют
данные по видео 43: отсутствуют
суммарное количество ошибок первого алгоритма на входящих  0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок первого алгоритма на выходящих 2
суммарное количество ошибок второго алгоритма на входящих  0
в видео 17 выходящих 6 а не 5
в видео 16 выходящих 4 а не 3
суммарное количество ошибок второго алгоритма на выходящих 2


In [ ]:
# т.к. словари при подгонке размерности видоизменяются создаем новые что бы не потерять старые 
d_true_helmet_exp = {str(n): [] for n in list(range(1,44))}
d_helmet_exp = {str(n): [] for n in list(range(1,44))}  
d_true_unif_exp = {str(n): [] for n in list(range(1,44))} 
d_unif_exp = {str(n): [] for n in list(range(1,44))} 

# Проходимся по всем словарям с нарушениями и эталонным словарям и выравниваем их
for N in range(1,44):
  d_helmet_exp[f'{N}'], d_true_helmet_exp[f'{N}'] = dict_fill(d_helmet[f'{N}'], d_true_helmet[f'{N}']) # выравниваем количество (длины массивов) человек для каждого видео в предсказаниях и эталонах (описание функции выше)
  d_unif_exp[f'{N}'], d_true_unif_exp[f'{N}'] = dict_fill(d_unif[f'{N}'], d_true_unif[f'{N}']) # выравниваем количество (длины массивов) человек для каждого видео в предсказаниях и эталонах (описание функции выше)

#превращаем значения словарей сначала в список списков а потом в вектор для подачи в склерн
d_true_helmet_list = [element for innerList in list(d_true_helmet_exp.values()) for element in innerList] 
d_helmet_list = [element for innerList in list(d_helmet_exp.values()) for element in innerList] 
d_true_unif_list = [element for innerList in list(d_true_unif_exp.values()) for element in innerList] 
d_unif_list = [element for innerList in list(d_unif_exp.values()) for element in innerList] 

In [ ]:
""" 
1.)Если задача избегать ложных детекций нарушений (у нас нарушения это 0), 
значит для нас важно не классифицировать 1 как 0 - а это значит, что цель определить все 1 как 1 
несмотря сколько 0 будет определно как 1, что в свою очередь означает что целевая метрика - Recall по 1,
а не precision как ранее обсуждали
2.) С учетом того что у нас нарушения имеют значения 0, а average ='binary' ожидает полчить нарушения 
в классе 1, мы убираем этот режим и смотрим на метрику класса 1 (это означает что первый класс 
помещен в числитель метрик P R и метрики вычисляются по нему) - что равносильно бинарному подходу 
по опредедению R для нарушений которые были бы в классе 1
"""
precision, recall,_,_ = precision_recall_fscore_support(d_true_helmet_list, d_helmet_list)
precision2, recall2,_,_ = precision_recall_fscore_support(d_true_unif_list, d_unif_list)
print('R по каскам: ', round(recall[1],2))
print('P по каскам: ', round(precision[1],2))
print('R по униформе: ', round(recall2[1],2))
print('P по униформе: ', round(precision2[1],2))

  _warn_prf(average, modifier, msg_start, len(result))

  _warn_prf(average, modifier, msg_start, len(result))



R по каскам:  1.0
P по каскам:  1.0
R по униформе:  0.92
P по униформе:  0.98


In [ ]:
d_true_unif

{'1': [1, 1, 1],
 '2': [1, 1, 1],
 '3': [1, 0, 1],
 '4': [1, 0, 1, 1],
 '5': [],
 '6': [0, 0, 0, 0, 1],
 '7': [1, 0, 1, 0],
 '8': [0, 0, 0, 0, 0],
 '9': [1, 1],
 '10': [1, 1],
 '11': [0, 0, 1],
 '12': [0, 0, 0, 1],
 '13': [1, 1, 0, 0, 1, 1],
 '14': [1, 1, 1],
 '15': [0, 0],
 '16': [1, 1, 1, 0],
 '17': [0, 1, 0, 0, 1, 1],
 '18': [1, 1, 0, 0],
 '19': [1],
 '20': [],
 '21': [0, 0],
 '22': [0, 1, 0],
 '23': [0],
 '24': [1],
 '25': [0, 0],
 '26': [0],
 '27': [0, 1],
 '28': [0, 1],
 '29': [0, 0],
 '30': [0, 1, 1, 1],
 '31': [1, 1],
 '32': [],
 '33': [],
 '34': [],
 '35': [],
 '36': [],
 '37': [],
 '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 '39': [1],
 '40': [0],
 '41': [],
 '42': [0, 0],
 '43': []}

In [ ]:
d_unif

{'1': [1, 1, 1],
 '2': [1, 1, 1],
 '3': [1, 0, 1],
 '4': [1, 0, 1, 1],
 '5': [],
 '6': [0, 0, 0, 0, 0],
 '7': [0, 1, 0, 0],
 '8': [0, 0, 0, 0, 0],
 '9': [1, 1],
 '10': [1, 1],
 '11': [0, 0, 1],
 '12': [0, 0, 0, 1],
 '13': [1, 1, 0, 0, 1, 1],
 '14': [1, 1, 1],
 '15': [0, 0],
 '16': [1, 1, 0],
 '17': [0, 1, 0, 1, 1],
 '18': [1, 1, 0, 0],
 '19': [1],
 '20': [],
 '21': [0, 0],
 '22': [0, 1, 0],
 '23': [0],
 '24': [1],
 '25': [0, 0],
 '26': [0],
 '27': [0, 1],
 '28': [0, 1],
 '29': [0, 0],
 '30': [0, 1, 1, 1],
 '31': [1, 1],
 '32': [],
 '33': [],
 '34': [],
 '35': [],
 '36': [],
 '37': [],
 '38': [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 '39': [1],
 '40': [0],
 '41': [],
 '42': [0, 0],
 '43': []}

In [ ]:
# отсутсвие нарушения 0 - целевой класс, мы не должны ошибаться на примерах с отсутсвующим нарушением
# фиксация нарушения 1

d_pred = [0,0,0,0,1,1,1,1,1,1] # одна ложная фиксация нарушения вместо фиксации отсутсвия
d_true = [0,0,0,0,0,1,1,1,1,1] 
cls_1 = classification_report(d_true, d_pred)
print('Одна ложная фиксация нарушения вместо фиксации его отстсвия (см. метрики класса 0)')
print('recall снизился, precision = 1')
print(cls_1)

d_pred = [0,1,1,1,1,1,1,1,1,1] 
d_true = [0,0,0,0,0,1,1,1,1,1] # четыре ложны фиксации нарушения вместо 4 фиксаций отстсвия
cls_2 = classification_report(d_true, d_pred)
print('четыре ложны фиксации нарушения вместо 4 фиксаций отстсвия (см. метрики класса 0)')
print('Все еще precision = 1, снижается precision нецелевого 1го класса')
print(cls_2)

In [ ]:
from sklearn.metrics import classification_report
# отсутсвие нарушения 0 - целевой класс, мы не должны ошибаться на примерах с отсутсвующим нарушением
# фиксация нарушения 1

d_pred = [0,0,0,0,0,1,1,1,1,1] 
d_true = [0,1,1,1,1,1,1,1,1,1] # пропуск 4 нарушений - допустимо в рамках задачи
cls_2 = classification_report(d_true, d_pred)
print('пропуск 4 нарушений - допустимо в рамках задачи')
print('Все еще precision = 0.2, recall = 1!')
print(cls_2)

пропуск 4 нарушений - допустимо в рамках задачи
Все еще precision = 0.2, recall = 1!
              precision    recall  f1-score   support

           0       0.20      1.00      0.33         1
           1       1.00      0.56      0.71         9

    accuracy                           0.60        10
   macro avg       0.60      0.78      0.52        10
weighted avg       0.92      0.60      0.68        10



In [ ]:
# отсутсвие нарушения 0 - целевой класс, мы не должны ошибаться на примерах с отсутсвующим нарушением
# фиксация нарушения 1

d_pred = [1,1,1,1,1,0,0,0,0,0] 
d_true = [1,0,0,0,0,0,0,0,0,0] # пропуск 4 нарушений - допустимо в рамках задачи

from sklearn.metrics import precision_recall_fscore_support
precision, recall,_,_ = precision_recall_fscore_support(d_true, d_pred)
print(precision)
print(recall)

[          1         0.2]
[    0.55556           1]


In [ ]:
print(d_true_helmet['7'])
print(d_true_helmet['11'])
print(d_true_helmet['13'])
print(d_true_helmet['14'])
print(d_true_helmet['17'])
print(d_true_helmet['30'])

[0, 0, 0, 0]
[0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 1, 1]
[0, 1, 0, 0, 0, 0]
[0, 0, 0]


In [ ]:
print(d_true_helmet_exp['7'])
print(d_true_helmet_exp['11'])
print(d_true_helmet_exp['13'])
print(d_true_helmet_exp['14'])
print(d_true_helmet_exp['17'])
print(d_true_helmet_exp['30'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
d_helmet_exp
print(d_helmet_exp['7'])
print(d_helmet_exp['11'])
print(d_helmet_exp['13'])
print(d_helmet_exp['14'])
print(d_helmet_exp['17'])
print(d_helmet_exp['30'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0]
[1, 1, 1, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]


# Гитхаб и ООП 

## mikel-brostrom

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git  # clone recursively
!cd yolov8_tracking
!pip install -r /content/yolov8_tracking/requirements.txt  # install dependencies

Cloning into 'yolov8_tracking'...
remote: Enumerating objects: 4640, done.
remote: Total 4640 (delta 0), reused 0 (delta 0), pack-reused 4640
Receiving objects: 100% (4640/4640), 109.43 MiB | 40.57 MiB/s, done.
Resolving deltas: 100% (2384/2384), done.
Submodule 'yolov8' (https://github.com/ultralytics/ultralytics) registered for path 'yolov8'
Cloning into '/content/OC_SORT/yolov8_tracking/yolov8'...
remote: Enumerating objects: 6287, done.        
remote: Counting objects: 100% (206/206), done.        
remote: Compressing objects: 100% (133/133), done.        
remote: Total 6287 (delta 108), reused 129 (delta 73), pack-reused 6081        
Receiving objects: 100% (6287/6287), 5.06 MiB | 19.99 MiB/s, done.
Resolving deltas: 100% (4226/4226), done.
Submodule path 'yolov8': checked out '15b3b0365ab2f12993a58985f3cb7f2137409a0c'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/yolov8_tracking/requirements.txt'


In [ ]:
# Импортируем библиотеки
import os
import pandas as pd
import cv2

In [ ]:
# Создаем папку weights в папке репозитория (требуется при запуске трекера)
!mkdir /content/yolov8_tracking/weights

mkdir: cannot create directory ‘/content/yolov8_tracking/weights’: No such file or directory


In [ ]:
# Проверяем наличие папки weights в папке репозитория
!ls /content/yolov8_tracking/

ls: cannot access '/content/yolov8_tracking/': No such file or directory


In [ ]:
# Запускаем трекер
!python /content/yolov8_tracking/track.py --yolo-weights /content/drive/MyDrive/AI/Inter_Yolo/best.pt/best14.pt --tracking-method bytetrack --tracking-config /content/bytetrack.yaml --source /content/drive/MyDrive/dataset-v1.1/test/14.mp4 --save-txt --save-conf --save-trajectories --save-vid


python: can't open file '/content/yolov8_tracking/track.py': [Errno 2] No such file or directory


In [ ]:
!git clone --recurse-submodules https://github.com/mf1024/ImageNet-Datasets-Downloader.git


Cloning into 'ImageNet-Datasets-Downloader'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 127 (delta 4), reused 6 (delta 2), pack-reused 113
Receiving objects: 100% (127/127), 1.58 MiB | 8.86 MiB/s, done.
Resolving deltas: 100% (68/68), done.


## ООП

##install and import

In [ ]:
!pip install ultralytics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html!

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html!


In [ ]:
import operator

import numpy as np
import torch
from ultralytics import YOLO
from ultralytics.tracker import BOTSORT
from ultralytics.tracker import BOTSORT
from ultralytics.tracker.track import on_predict_start
# from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics.yolo.utils.checks import check_requirements
# from ultralytics.yolo.utils.plotting import save_one_box, colors
from ultralytics.yolo.v8.detect import DetectionPredictor
from collections import defaultdict

requirements: YOLOv8 requirement "lap>=0.4" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lap: filename=lap-0.4.0-cp39-cp39-linux_x86_64.whl size=1655021 sha256=90361ce8d202466847b7c045f37e175a5f7b18cf81aaabd84e72075f40892c45
  Stored in directory: /root/.cache/pip/wheels/2f/8b/30/e7dd4f9dc44fb438381df571c9a6bddc35aafd1bf39c4f8911
Successfully built lap

requirements: 1 package updated per ['lap>=0.4']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [ ]:
check_requirements('lap')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Detect and post-processing

In [ ]:
class MyDetectionPredictor2(DetectionPredictor):
    track_id_barrier = defaultdict(list)
    barrier_y = 370
    track_dict = {}
    barrier_zone = 5

    @staticmethod
    def get_coord_y(coord_y, last_y):
        return coord_y - last_y > 0

    @staticmethod
    def get_y_center(arr):
        center_y = ((abs((arr[:, 3] - arr[:, 2])) / 2) + arr[:, 2])
        return center_y

    def get_control_walk(self, last_y, id_human, y_coord):
        operator_walk, text = (operator.gt, 'Вышел') if last_y > 0 else (operator.lt, 'Вошел')

        if operator_walk(self.barrier_y, y_coord):
            print('+' * 100)
            print('id_human', id_human, text)
            print('+' * 100)
            self.track_dict.pop(id_human, None)

    def write_results(self, idx, results, batch):
        log_string = ''
        tracks = []
        det_track = results[idx].boxes.cpu().numpy()
        det_track_not_human = det_track[det_track.cls != 0]
        det_track_human = det_track[det_track.cls == 0]
        if det_track_human:
            self.results[idx].update(boxes=torch.as_tensor(det_track_human.boxes))
            self.res_coord = [coord for coord in det_track_human.boxes]

        im0s = self.batch[2]
        im0s = im0s if isinstance(im0s, list) else [im0s]

        if hasattr(self, 'trackers'):
            tracks = self.trackers[0].update(det_track_human, im0s[0])

        if len(tracks) == 0:
            return super().write_results(idx, results, batch)
        ########################################################################
        id_array = tracks[:, 4]

        y = self.get_y_center(tracks)

        tracks_array = tracks[np.in1d(tracks[:, 4], list(self.track_dict.keys()))]
        for y_coord, id_human in zip(self.get_y_center(tracks_array), tracks_array[:, 4]):
            last_y = self.track_dict.get(id_human)
            self.get_control_walk(last_y, id_human, y_coord)

        for c_y, id_human in zip(y, id_array):
            dst = c_y - self.barrier_y
            if abs(dst) < self.barrier_zone:#От барьера 5 в обе стороны
                self.track_dict.update({id_human: dst})
        ########################################################################

        self.results[idx].update(boxes=torch.as_tensor(tracks[:, :-1]))
        log_string += super().write_results(idx, self.results, batch)

        return log_string



In [ ]:
def init_model(track=True, model="yolov8s.pt"):
    my_model = YOLO(model=model)
    if track:
        my_model.predictor = MyDetectionPredictor2()
    return my_model

In [ ]:
def predict_model(source=None, **kwargs):
    event = 'on_predict_start'
    my_model = init_model()
    # kwargs = {}
    # conf = 0.1
    # kwargs['conf'] = conf
    kwargs['mode'] = 'track'
    my_model.add_callback(event, on_predict_start)
    result = my_model.predict(source=source, **kwargs)
    return result



result = predict_model(source='1.mp4', save=True, classes=0, line_thickness=1)
#По 4 посетителю середина (Y) прыгнула поэтому и зашел и вышел 

100%|██████████| 21.5M/21.5M [00:00<00:00, 107MB/s]

Ultralytics YOLOv8.0.68 🚀 Python-3.9.16 torch-2.0.0+cu118 CPU
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


FileNotFoundError: ignored

In [ ]:
data[0]

In [ ]:

for key, value in data[1].items():
  print(f'Нарушитель с id-{key} ')
  for v in value:
      
      print(f'номер нарушения-{v[0]}, момент нарушения -{v[1].split()[:-1]}')


### Track


In [ ]:
def init_track_model():
    my_model = YOLO()
    my_model.track(source='/content/gdrive/MyDrive/My_dataset/dataset-v1.1/test/1.mp4', save=True, save_txt=True)
    return my_model

In [ ]:
init_track_model()

## Dimar

In [ ]:
# Подключаем диск: на диске видео и потом туда сохраняем labels и видео с результатами.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!git clone https://github.com/dimarsoft/yolov7.git
%cd yolov7
!pip install -r requirements.txt  # install dependencies

Cloning into 'yolov7'...
remote: Enumerating objects: 2240, done.
remote: Counting objects: 100% (568/568), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 2240 (delta 379), reused 565 (delta 376), pack-reused 1672
Receiving objects: 100% (2240/2240), 143.21 MiB | 23.79 MiB/s, done.
Resolving deltas: 100% (1352/1352), done.
Updating files: 100% (248/248), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.0/511.0 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepar

**Трекинг по уже полученным тхт файлам**

Так же можно реализовать свой фукцию изменения bbox для трекера




In [ ]:
# ничего не меняет, просто print
def not_change_bbox(bbox, file_id):
    """

    Args:
        file_id(str): имя файла 
        bbox (tensor): первые 4ре столбца x1 y1 x2 y2 
        в абсолютных значениях картинки.
        Пока это ббоксы всех классов, 
        сам класс находится в последнем столбце (-1, или индекс 5)
    """
    print(f"file = {file_id}")
    return bbox

# пример реалиции бокса по центру 20/20
def change_bbox_to_center(bbox, file_id):
    x1 = (bbox[:, [0]] + bbox[:, [2]]) / 2
    y1 = (bbox[:, [1]] + bbox[:, [3]]) / 2

    w = 10  
    h = 10  

    bbox[:, [0]] = x1 - w
    bbox[:, [2]] = x1 + w

    bbox[:, [1]] = y1 - h
    bbox[:, [3]] = y1 + h

    return bbox    

# пример реалиции от Павла (группа №1)
def pavel_change_bbox(bbox, file_id):
    y2 = bbox[:, [1]] + 150

    bbox[:, [3]] = y2

    return bbox

In [ ]:
from yolo_track_by_txt import run_track_yolo
from configs import get_all_trackers_full_path

# путь к файлу/папке, если указать путь к папке, то обработает все видео
video_source = "/content/drive/MyDrive/dataset-v1.1/test" 


files= None # ["1", "2"] # None # ['1']
files= ["1"] # None # ['1']

# classes спискок классов. [0] # [0, 1] # список классов, None все. 0 - человек. 
# это нужно к примеру когда используется предобученная модель, в которой много классов, но нам нужен только один человек
# или хотим посмотреть когда трекер только по одному классу работает
classes = None
# создание видео 
save_vid=True
save_txt=True

change_bb = None # change_bbox_to_center

# путь к текстовым файла (папка)
txt_source_folder = "/content/drive/MyDrive/AI_2023/2023_03_27_15_05_37_YoloVersion.yolo_v8_detect"
txt_source_folder = "/content/drive/MyDrive/AI_2023/Detections/2023_03_29_10_35_01_YoloVersion.yolo_v7_detect"
# папка для сохранения информации, пока тхт, потом наше видео, постобработка
# в ней создается папка с датой, там будут видео появляться, тхт файлы и в конце результата постобработки и сравнения с эталоном
output_folder = "/content/drive/MyDrive/AI_2023/"

reid_weights = "osnet_x0_25_msmt17.pt"
test_file = "/content/yolov7/testinfo/all_track_results.json"



# в репозитории есть именные фукции постобработки: "timur", "popov_alex", "dimar"
# если указать строку, то будет выполняться код из репозитория
# post_processing/alex.py, post_processing/timur.py
# в функции run_track_yolo есть параметр test_func: test_func=имя фукции для постобработки
# имя функции для постобработки может быть любое , допустимое в python

test_func = None


# все доступные трекеры
all_trackers = get_all_trackers_full_path()

# пример для norfair
tracker_name = "norfair"
tracker_config = all_trackers.get(tracker_name)

# или как ранее
# tracker_config = "/content/yolov7/trackers/NorFairTracker/configs/norfair_track.yaml"

# 1. запуск одного трекера
run_track_yolo(txt_source_folder, video_source, tracker_name, tracker_config,
               output_folder, reid_weights, test_file, test_func=test_func, files=files, 
               save_vid=save_vid, change_bb=change_bb)

# список доступных трекеров

for key in all_trackers.keys():
  print(f"{key} : {all_trackers[key]}")


# 2. запуск всех трекеров , но в одной сессии: папка одна бедт общпя для всех трекеров

# указываем словарь, run_track_yolo по нему отработает
tracker_name = all_trackers

run_track_yolo(txt_source_folder, video_source, tracker_name, tracker_config,
               output_folder, reid_weights, test_file, test_func=test_func, files=files, 
               save_vid=save_vid, change_bb=change_bb)
  